In [19]:
# FUNCTION DECLARATIONS

# getAspectDescription(text: string) => [{aspect: string, description: string}]
# getSentiment(raw_text: string) => (output: string, prediction: (polarity, subjectivity))

# getAspectDescription

In [20]:
# %pip install spacy

In [21]:
# We get started by importing spacy
import spacy
nlp = spacy.load("en_core_web_lg")

In [22]:
# getAspectDescription(text: string) => [{aspect: string, description: string}]

def getAspectDescription(text):
    # getAspectDescription(text: string) => [{aspect: string, description: string}]
    aspects = []

    doc = nlp(text)
    descriptive_term = ''
    target = []
    for token in doc:
        if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
            # target = token.text
            target.append(token.text)
        if token.pos_ == 'ADJ':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            descriptive_term = f"{prepend}" + token.text

    aspects.append({'aspect': target, 'description': descriptive_term})

    return aspects


In [23]:
sentences = [
    "The sound quality is amazing and the earphones are very comfortable to wear. I would definitely recommend this product to anyone looking for a high-quality wireless earphone.",
    "These earphones are a great value for the price. They have a long battery life and the sound quality is really good. I'm very happy with my purchase.",
    "I'm really impressed with these earphones. They connect to my phone really quickly and the sound quality is excellent. Plus, they're very lightweight and comfortable to wear.",
    "These earphones are so convenient! I love that they're wireless and I don't have to worry about cords getting tangled. The sound quality is great too.",
    "I bought these earphones for working out and they're perfect. They stay in place really well and the sound quality is really good. I would definitely recommend them to anyone looking for wireless earphones for working out.",
    "These earphones are amazing! The sound quality is excellent and they're really comfortable to wear. Plus, the battery life is really impressive - I can use them for hours without having to recharge.",
    "I'm really happy with these earphones. They fit my ears really well and the sound quality is excellent. Plus, they're really easy to use - I can control the volume and playback from the earphones themselves.",
    "These earphones are so stylish! I love the sleek design and the sound quality is really good too. They're also really easy to pair with my phone.",
    "I'm very impressed with these earphones. They have a great range and I can use them all over my house without losing connection. The sound quality is also really good.",
    "These earphones are a great value for the price. They have a long battery life and the sound quality is really good. I would definitely recommend them to anyone looking for wireless earphones."
]

for sentence in sentences:
    print(getAspectDescription(sentence))

[{'aspect': ['quality', 'earphones'], 'description': 'wireless'}]
[{'aspect': ['earphones', 'quality'], 'description': 'very happy'}]
[{'aspect': ['quality'], 'description': 'comfortable'}]
[{'aspect': ['earphones', 'quality'], 'description': 'great'}]
[{'aspect': ['quality'], 'description': 'wireless'}]
[{'aspect': ['earphones', 'quality', 'life'], 'description': 'really impressive'}]
[{'aspect': ['quality'], 'description': 'really easy'}]
[{'aspect': ['earphones', 'quality'], 'description': 'really easy'}]
[{'aspect': ['quality'], 'description': 'really good'}]
[{'aspect': ['earphones', 'quality'], 'description': 'wireless'}]


# getSentiment

In [24]:
# %pip install pandas
# %pip install re
# %pip install bs4
# %pip install nltk
# %pip install pickle

In [25]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pickle

In [28]:
# getSentiment(raw_text: string) => (output: string, prediction: (polarity, subjectivity))

model = pickle.load(open("C:/Users/acer/Documents/GitHub/ASPECTandSENTIMENT/modelNB.pkl", 'rb'))

def getSentiment(raw_text: str):

    # Instantiate PorterStemmer
    p_stemmer = PorterStemmer()

    # Remove HTML
    review_text = BeautifulSoup(raw_text).get_text()

    # Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)

    # Convert words to lower case and split each word up
    words = letters_only.lower().split()

    # Convert stopwords to a set
    stops = set(stopwords.words('english'))

    # Adding on stopwords that were appearing frequently in both positive and negative reviews
    stops.update(['app','shopee','shoppee','item','items','seller','sellers','bad'])

    # Remove stopwords
    meaningful_words = [w for w in words if w not in stops]

    # Stem words
    meaningful_words = [p_stemmer.stem(w) for w in meaningful_words]

    # Join words back into one string, with a space in between each word
    final_text = pd.Series(" ".join(meaningful_words))

    # Generate predictions
    pred = model.predict(final_text)[0]
    probability = model.predict_proba([pd.Series.to_string(final_text)])
    # text = 'The product is strong and great!'
# probs = gs_tvec_nb.predict_proba([text])  # predict probabilities for the text
# print('Predicted probabilities:', probs)  # print the predicted probabilities

    if pred == 1:
        output = "Negative"
    else:
        output = "Postive"

    return output, probability


# text = """
# anc work in the first 3 days but the proximity sensor on the left bud stopped working
# """
# result = getSentiment(text)
# print(f'RAW_TEXT: \t{text}')
# print(F'\tSENTIMENT: {result}')

In [29]:
text = 'The product is strong, but it was not the right color. Poor packaging'

In [30]:
output, proba = getSentiment(text)

In [31]:
print(f"Text: {text}", output, proba, sep='\t')

Text: The product is strong, but it was not the right color. Poor packaging	Negative	[[0.22424745 0.77575255]]


# Aspect Sentiment Classification

In [32]:
sentences = [
    "The sound quality is amazing and the earphones are very comfortable to wear. I would definitely recommend this product to anyone looking for a high-quality wireless earphone.",
    "These earphones are a great value for the price. They have a long battery life and the sound quality is really good. I'm very happy with my purchase.",
    "I'm really impressed with these earphones. They connect to my phone really quickly and the sound quality is excellent. Plus, they're very lightweight and comfortable to wear.",
    "These earphones are so convenient! I love that they're wireless and I don't have to worry about cords getting tangled. The sound quality is great too.",
    "I bought these earphones for working out and they're perfect. They stay in place really well and the sound quality is really good. I would definitely recommend them to anyone looking for wireless earphones for working out.",
    "These earphones are amazing! The sound quality is excellent and they're really comfortable to wear. Plus, the battery life is really impressive - I can use them for hours without having to recharge.",
    "I'm really happy with these earphones. They fit my ears really well and the sound quality is excellent. Plus, they're really easy to use - I can control the volume and playback from the earphones themselves.",
    "These earphones are so stylish! I love the sleek design and the sound quality is really good too. They're also really easy to pair with my phone.",
    "I'm very impressed with these earphones. They have a great range and I can use them all over my house without losing connection. The sound quality is also really good.",
    "These earphones are a great value for the price. They have a long battery life and the sound quality is really good. I would definitely recommend them to anyone looking for wireless earphones."
]

aspectSentiments = []
for sentence in sentences:
    aspects = getAspectDescription(sentence)
    aspectSentiment = [{'aspect': aspect['aspect'], 'description': aspect['description'], 'sentiment': getSentiment(sentence)} for aspect in aspects]
    aspectSentiments.extend(aspectSentiment)

for item in aspectSentiments:
    print(f"Aspect: {item['aspect']}\nDescription: {item['description']}\nSentiment: {item['sentiment']}\n")

Aspect: ['quality', 'earphones']
Description: wireless
Sentiment: ('Postive', array([[0.87712357, 0.12287643]]))

Aspect: ['earphones', 'quality']
Description: very happy
Sentiment: ('Postive', array([[0.91932053, 0.08067947]]))

Aspect: ['quality']
Description: comfortable
Sentiment: ('Postive', array([[0.30406668, 0.69593332]]))

Aspect: ['earphones', 'quality']
Description: great
Sentiment: ('Postive', array([[0.96336511, 0.03663489]]))

Aspect: ['quality']
Description: wireless
Sentiment: ('Postive', array([[0.60903319, 0.39096681]]))

Aspect: ['earphones', 'quality', 'life']
Description: really impressive
Sentiment: ('Postive', array([[0.94260008, 0.05739992]]))

Aspect: ['quality']
Description: really easy
Sentiment: ('Postive', array([[0.70346619, 0.29653381]]))

Aspect: ['earphones', 'quality']
Description: really easy
Sentiment: ('Postive', array([[0.92365788, 0.07634212]]))

Aspect: ['quality']
Description: really good
Sentiment: ('Postive', array([[0.96728605, 0.03271395]]))

# Aspect-Sentence Mapping

In [68]:
sentences = [
    "The phone has a huge display that is crystal clear. I love watching videos on it.",
    "This phone is compact and fits perfectly in my pocket. The size is just right.",
    "The camera on this phone is amazing. The pictures come out clear and vivid.",
    "The battery life on this phone is incredible. I can go all day without needing to charge it.",
    "I'm impressed with the phone's sleek design. It looks great and feels good in my hand.",
    "The phone's display is one of the best I've seen. The colors are vibrant and the screen is bright.",
    "The size of this phone is perfect for one-handed use. It's not too big or too small.",
    "The camera on this phone takes amazing low light photos. I'm really happy with the quality.",
    "The battery on this phone lasts all day and charges quickly. I'm never without a charged phone.",
    "I'm in love with the phone's display. It's so clear and bright, even in direct sunlight."
]

#keys = []

#for sentence in sentences:
    #aspects = getAspectDescription(sentence)
    #for aspect in aspects:
        #keys.append(str(aspect['aspect']))
#aspect_dict = {key: [] for key in keys}

#for item in aspectSentiments:
    #sentiment = item['sentiment']
    #for aspect in aspects:
        #if aspect not in aspect
    
#for key in keys:
    #print(key)


aspectSentiments = []
for sentence in sentences:
    aspects = getAspectDescription(sentence)
    aspectSentiment = [{'aspect': aspect['aspect'], 'description': aspect['description'], 'sentiment': getSentiment(sentence)} for aspect in aspects]
    aspectSentiments.extend(aspectSentiment)

for item in aspectSentiments:
    print(f"Aspect: {item['aspect']}\nDescription: {item['description']}\nSentiment: {item['sentiment']}\n")


Aspect: ['phone']
Description: clear
Sentiment: ('Postive', array([[0.59031395, 0.40968605]]))

Aspect: ['phone', 'size']
Description: just right
Sentiment: ('Negative', array([[0.15916132, 0.84083868]]))

Aspect: ['camera', 'pictures']
Description: vivid
Sentiment: ('Postive', array([[0.60490228, 0.39509772]]))

Aspect: ['life']
Description: incredible
Sentiment: ('Negative', array([[0.2054525, 0.7945475]]))

Aspect: []
Description: good
Sentiment: ('Postive', array([[0.71911786, 0.28088214]]))

Aspect: ['display', 'colors', 'screen']
Description: bright
Sentiment: ('Negative', array([[0.51080566, 0.48919434]]))

Aspect: ['size']
Description: too small
Sentiment: ('Negative', array([[0.26103637, 0.73896363]]))

Aspect: ['camera']
Description: really happy
Sentiment: ('Postive', array([[0.52485902, 0.47514098]]))

Aspect: []
Description: 
Sentiment: ('Negative', array([[0.26878781, 0.73121219]]))

Aspect: []
Description: direct
Sentiment: ('Postive', array([[0.59031395, 0.40968605]]))


In [66]:
#import json

Craig

In [64]:
sentences = [
    "The phone has a huge display that is crystal clear. I love watching videos on it.",
    "This phone is compact and fits perfectly in my pocket. The size is just right.",
    "The camera on this phone is amazing. The pictures come out clear and vivid.",
    "The battery life on this phone is incredible. I can go all day without needing to charge it.",
    "I'm impressed with the phone's sleek design. It looks great and feels good in my hand.",
    "The phone's display is one of the best I've seen. The colors are vibrant and the screen is bright.",
    "The size of this phone is perfect for one-handed use. It's not too big or too small.",
    "The camera on this phone takes amazing low light photos. I'm really happy with the quality.",
    "The battery on this phone lasts all day and charges quickly. I'm never without a charged phone.",
    "I'm in love with the phone's display. It's so clear and bright, even in direct sunlight."
]

#keys = []

#for sentence in sentences:
    #aspects = getAspectDescription(sentence)
    #for aspect in aspects:
        #keys.append(str(aspect['aspect']))
#aspect_dict = {key: [] for key in keys}

#for item in aspectSentiments:
    #sentiment = item['sentiment']
    #for aspect in aspects:
        #if aspect not in aspect
    
#for key in keys:
    #print(key)


aspect_dict = {}
aspects_arr = []
sentiments_arr = []

for item in aspectSentiments:
    aspects_arr.append(tuple(item['aspect']))
    sentiments_arr.append(item['sentiment'])

for i in range(len(aspects_arr)):
    aspect_dict[aspects_arr[i]] = {
        'sentence': sentences[i], 
        'sentiment': sentiments_arr[i] 
    }


In [69]:
#print(aspects_arr)
#print(sentiments_arr)
#print(description_arr)

for key, value in aspect_dict.items():
    print(key)
    print(value)

('phone',)
{'sentence': 'The phone has a huge display that is crystal clear. I love watching videos on it.', 'sentiment': ('Postive', array([[0.59031395, 0.40968605]]))}
('phone', 'size')
{'sentence': 'This phone is compact and fits perfectly in my pocket. The size is just right.', 'sentiment': ('Negative', array([[0.15916132, 0.84083868]]))}
('camera', 'pictures')
{'sentence': 'The camera on this phone is amazing. The pictures come out clear and vivid.', 'sentiment': ('Postive', array([[0.60490228, 0.39509772]]))}
('life',)
{'sentence': 'The battery life on this phone is incredible. I can go all day without needing to charge it.', 'sentiment': ('Negative', array([[0.2054525, 0.7945475]]))}
()
{'sentence': "I'm in love with the phone's display. It's so clear and bright, even in direct sunlight.", 'sentiment': ('Postive', array([[0.59031395, 0.40968605]]))}
('display', 'colors', 'screen')
{'sentence': "The phone's display is one of the best I've seen. The colors are vibrant and the scre